In [59]:
import numpy as np
import pandas as pd
import re

from collections import Counter
import string


from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter,Okt

import os

from tqdm import tqdm

import kss

In [60]:
okt = Okt()

In [61]:
path_ = r"C:\Users\user\Documents\DL_project\ear_phone"
file_list = os.listdir(path_)
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
file_list_xlsx

['air.xlsx', 'air_pro.xlsx', 'buzz.xlsx', 'buzz_live.xlsx', 'buzz_plus.xlsx']

In [63]:
for f in file_list_xlsx:
    file_path = path_ + "\\" + f
    name = f.replace(".xlsx","")
    globals () ["df_{}".format(name)] =  pd.read_excel(file_path).drop(columns = "Unnamed: 0")
    globals () ["df_{}".format(name)]["contents"] =  globals () ["df_{}".format(name)]["contents"].map(lambda x : str(x).replace("None",""))

In [64]:
def remove_null(df_,col_):
    return df_[df_[col_] != ""].reset_index(drop=True).sort_values(by=["liked"],ascending=False)

In [65]:
def sort_len(df_):
    df_1 = df_[df_["liked"]!=0]
    df_2 = df_[df_["liked"]==0]
    df_2["len"] = df_2["contents"].map(lambda x :len(x))
    df_2 = df_2.sort_values(by=["len"],ascending=False)
    df_3 = df_2.drop(columns="len")
    return pd.concat([df_1,df_3]).reset_index(drop=True)

In [66]:
def enter_split(df_):
    enter_ = []
    enter_label = []
    for i in range(len(df_)):
        split_ = df_.loc[i,"contents"].split("\n")
        try:
            split_.remove("")
        except:
            pass
        split_len = len(split_)
        enter_.extend(split_)
        enter_label.extend([i]*split_len)
    df_2 = pd.DataFrame([enter_,enter_label]).T
    df_2 = df_2.rename(columns={0:"enter",1:"review"})
    return df_2[df_2["enter"] != ""].reset_index(drop=True)

In [67]:
def sent_split(df_):
    print("-"*40,"sentence split","-"*40)
    sen_li = []
    review_num = []
    enter_num = []
    for i in tqdm(range(len(df_))):
        sen = kss.split_sentences(df_["enter"][i])
        sen_len = len(sen)
        rev_num = df_.loc[i,"review"]
        sen_li.extend(sen)
        review_num.extend([rev_num]*sen_len)
        enter_num.extend([i]*sen_len)
    df_2 = pd.DataFrame([sen_li,review_num,enter_num]).T.reset_index(drop=True)
    df_2 = df_2.rename(columns={0:"sentence",1:"review",2:"enter"})
    #df_["words"] = words
    return df_2

In [68]:
def pos_split(df_):
    print("-"*40,"pos tagging","-"*40)
    word_li = []
    pos_li = []
    
    re_li = []
    enter_li = []
    sen_li = []
    for i in tqdm(range(len(df_))):
        #print(df_["sentence"][i])
        try:
            word_pos = okt.pos(df_["sentence"][i],stem=True)
            wp_len = len(word_pos)
            word_ = [w[0] for w in word_pos]
            pos_ = [w[1] for w in word_pos]

            word_li.extend(word_)
            pos_li.extend(pos_)

            re_li.extend([df_.loc[i,"review"]]*wp_len)
            enter_li.extend([df_.loc[i,"enter"]]*wp_len)
            sen_li.extend([i]*wp_len)
        except:
            print(df_["sentence"][i])
    df_2 = pd.DataFrame([word_li,sen_li,enter_li,re_li,pos_li]).T.reset_index(drop=True)
    df_2 = df_2.rename(columns={0:"word",1:"sentence",2:"enter",3:"review",4:"pos"})
    return df_2

In [69]:
def make_voca(voca_):
    voca = {}
    for i in range(len(voca_)):
        fea = voca_.loc[i,"feature"]
        word = voca_.loc[i,"word"].split(",")
        voca[fea] = word
    voca_all = []
    for i in list(voca.values()):
        voca_all.extend(i)
    return voca,voca_all

In [70]:
def find_value(vo,value):
    for i in vo.keys():
        if value in vo[i]:
            return i

In [71]:
def rule_marking(df_,voca,voca_all):
    mark_li = []
    for i in range(len(df_)):
        word_ = df_.loc[i,"word"]
        if word_ in voca_all:
            fe = find_value(voca,word_)
            mark_li.append(fe)
        else:
            mark_li.append("O")
    df_["rule_label"] = mark_li
    #df_2 = [pd.Dataframe(list(df_["word"])),mark_li,df_.drop(columns="word")]
    return df_

In [72]:
test = df_air
voca_ = pd.read_excel(r"C:\Users\user\Documents\DL_project\Vocabulary_final.xlsx",sheet_name="에어팟")

In [73]:
test_re = remove_null(test,"contents")
test_re2 = sort_len(test_re)
test_enter = enter_split(test_re2)
test_sen = sent_split(test_enter)
test_word =pos_split(test_sen)
voca,voca_all = make_voca(voca_)
test_rule = rule_marking(test_word,voca,voca_all)
test_rule

<ipython-input-65-df78ea830372>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["len"] = df_2["contents"].map(lambda x :len(x))
 10%|███████▌                                                                  | 6357/61783 [00:00<00:01, 31584.97it/s]

---------------------------------------- sentence split ----------------------------------------


  0%|▏                                                                           | 241/86967 [00:00<01:11, 1218.66it/s]

---------------------------------------- pos tagging ----------------------------------------


100%|██████████████████████████████████████████████████████████████████████████| 86967/86967 [00:54<00:00, 1581.65it/s]


,word,sentence,enter,review,pos,rule_label
0,●,0,0,0,Foreign,O
1,구매,0,0,0,Noun,O
2,동기,0,0,0,Noun,O
3,에어팟,1,1,0,Noun,O
4,!!!!!!!!!!!!!!!!!!!!!!!!!!!,1,1,0,Punctuation,O
...,...,...,...,...,...,...
691491,굿,86962,61778,38045,Noun,O
691492,굿,86963,61779,38046,Noun,O
691493,굿,86964,61780,38047,Noun,O
691494,굿,86965,61781,38048,Noun,O


In [52]:
test_rule.to_excel(r"C:\Users\user\Documents\DL_project\NER\air.xlsx")

In [53]:
test = df_air_pro
voca_ = pd.read_excel(r"C:\Users\user\Documents\DL_project\Vocabulary_final.xlsx",sheet_name="에어팟프로")

In [54]:
test_re = remove_null(test,"contents")
test_re2 = sort_len(test_re)
test_enter = enter_split(test_re2)
test_sen = sent_split(test_enter)
test_word =pos_split(test_sen)
voca,voca_all = make_voca(voca_)
test_rule = rule_marking(test_word,voca,voca_all)
test_rule

<ipython-input-34-df78ea830372>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["len"] = df_2["contents"].map(lambda x :len(x))
 14%|██████████▎                                                               | 6250/44656 [00:00<00:01, 30637.16it/s]

---------------------------------------- sentence split ----------------------------------------


  0%|▎                                                                           | 240/62304 [00:00<00:52, 1174.10it/s]

---------------------------------------- pos tagging ----------------------------------------


100%|██████████████████████████████████████████████████████████████████████████| 62304/62304 [00:43<00:00, 1444.73it/s]


,word,sentence,enter,review,pos,rule_label
0,구,0,0,0,Modifier,O
1,매일,0,0,0,Noun,O
2,7월,0,0,0,Number,O
3,2일,0,0,0,Number,O
4,오후,0,0,0,Noun,O
...,...,...,...,...,...,...
560927,ㄷ,62299,44651,21861,KoreanParticle,O
560928,국,62300,44652,21862,Noun,O
560929,굳다,62301,44653,21863,Adjective,O
560930,굿,62302,44654,21864,Noun,O


In [55]:
test_rule.to_excel(r"C:\Users\user\Documents\DL_project\NER\air_pro.xlsx")

In [56]:
test = df_buzz
voca_ = pd.read_excel(r"C:\Users\user\Documents\DL_project\Vocabulary_final.xlsx",sheet_name="버즈")

In [57]:
test_re = remove_null(test,"contents")
test_re2 = sort_len(test_re)
test_enter = enter_split(test_re2)
test_sen = sent_split(test_enter)
test_word =pos_split(test_sen)
voca,voca_all = make_voca(voca_)
test_rule = rule_marking(test_word,voca,voca_all)
test_rule

<ipython-input-34-df78ea830372>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["len"] = df_2["contents"].map(lambda x :len(x))
 13%|█████████▉                                                                | 2918/21632 [00:00<00:00, 28965.39it/s]

---------------------------------------- sentence split ----------------------------------------


  0%|▏                                                                             | 78/29609 [00:00<01:18, 378.30it/s]

---------------------------------------- pos tagging ----------------------------------------


100%|██████████████████████████████████████████████████████████████████████████| 29609/29609 [00:22<00:00, 1316.91it/s]


,word,sentence,enter,review,pos,rule_label
0,☆☆,0,0,0,Foreign,O
1,나름,0,0,0,Noun,O
2,음향,0,0,0,Noun,SQ
3,기기,0,0,0,Noun,O
4,좋아하다,0,0,0,Adjective,O
...,...,...,...,...,...,...
287830,굿,29604,21627,9490,Noun,O
287831,굿,29605,21628,9491,Noun,O
287832,굿,29606,21629,9492,Noun,O
287833,굿,29607,21630,9493,Noun,O


In [58]:
test_rule.to_excel(r"C:\Users\user\Documents\DL_project\NER\buds.xlsx")

In [74]:
test = df_buzz_plus
voca_ = pd.read_excel(r"C:\Users\user\Documents\DL_project\Vocabulary_final.xlsx",sheet_name="버즈")

In [75]:
test_re = remove_null(test,"contents")
test_re2 = sort_len(test_re)
test_enter = enter_split(test_re2)
test_sen = sent_split(test_enter)
test_word =pos_split(test_sen)
voca,voca_all = make_voca(voca_)
test_rule = rule_marking(test_word,voca,voca_all)
test_rule

<ipython-input-65-df78ea830372>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["len"] = df_2["contents"].map(lambda x :len(x))
 36%|██████████████████████████▋                                               | 6410/17744 [00:00<00:00, 31864.13it/s]

---------------------------------------- sentence split ----------------------------------------


  1%|▍                                                                           | 124/24419 [00:00<00:19, 1231.01it/s]

---------------------------------------- pos tagging ----------------------------------------


100%|██████████████████████████████████████████████████████████████████████████| 24419/24419 [00:18<00:00, 1356.07it/s]


,word,sentence,enter,review,pos,rule_label
0,<,0,0,0,Punctuation,O
1,상품,0,0,0,Noun,O
2,평,0,0,0,Noun,O
3,하단,0,0,0,Noun,O
4,에,0,0,0,Josa,O
...,...,...,...,...,...,...
228973,굿,24414,17739,8151,Noun,O
228974,굿,24415,17740,8152,Noun,O
228975,굿,24416,17741,8153,Noun,O
228976,굿,24417,17742,8154,Noun,O


In [76]:
test_rule.to_excel(r"C:\Users\user\Documents\DL_project\NER\buds_plus.xlsx")

In [77]:
test = df_buzz_live
voca_ = pd.read_excel(r"C:\Users\user\Documents\DL_project\Vocabulary_final.xlsx",sheet_name="버즈")

In [78]:
test_re = remove_null(test,"contents")
test_re2 = sort_len(test_re)
test_enter = enter_split(test_re2)
test_sen = sent_split(test_enter)
test_word =pos_split(test_sen)
voca,voca_all = make_voca(voca_)
test_rule = rule_marking(test_word,voca,voca_all)
test_rule

<ipython-input-65-df78ea830372>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["len"] = df_2["contents"].map(lambda x :len(x))
100%|███████████████████████████████████████████████████████████████████████████| 3888/3888 [00:00<00:00, 33328.67it/s]

---------------------------------------- sentence split ----------------------------------------



  2%|█▎                                                                             | 83/5190 [00:00<00:12, 398.66it/s]

---------------------------------------- pos tagging ----------------------------------------


100%|████████████████████████████████████████████████████████████████████████████| 5190/5190 [00:04<00:00, 1199.38it/s]


,word,sentence,enter,review,pos,rule_label
0,☆☆,0,0,0,Foreign,O
1,나름,0,0,0,Noun,O
2,음향,0,0,0,Noun,SQ
3,기기,0,0,0,Noun,O
4,좋아하다,0,0,0,Adjective,O
...,...,...,...,...,...,...
58852,굿,5185,3883,1334,Noun,O
58853,굿,5186,3884,1335,Noun,O
58854,굿,5187,3885,1336,Noun,O
58855,굿,5188,3886,1337,Noun,O


In [79]:
test_rule.to_excel(r"C:\Users\user\Documents\DL_project\NER\buds_live.xlsx")